In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Saul\AppData\Local\Temp\ipykernel_8516\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speed up development. 

In [4]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


In [5]:
data.sample(20)

,text,label
727,In view of the yearly sweepstake of the Onlae ...,1
281,FROM THE DESK OF AZZEM YARO.BILL AND EXCHANGE ...,1
959,Coll=E8gue=2CJ'ai recu une bonne informatio...,1
300,peterjohnsonlawfirm0014@hotmail.comYou have a ...,1
92,CheckingI am sure they willMitchell just respo...,0
125,<HTML><BODY><div><div><TABLE cellSpacing=0 cel...,1
282,DR TONY COLEFAX=3B 234 1 7599159EMAIL=3A tony3...,1
777,Sorry - first I've heard of it. Philippe and I...,0
286,RE CONFIDENTIALDear Sir=2FCeo=2CI am=2C DR=2EL...,1
18,"Abidjan Cote D'Ivoire=20Dear,=20It is my pleas...",1


## Data Preprocessing

In [6]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
stop_words = stopwords.words('english')
print(stop_words[100:110])

from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [7]:
# Your code
import re

def remove_inline_js_css(text):
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL) # Regex to remove <script>...</script> tags and their content
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL) # Regex to remove <style>...</style> tags and their content
    return text

def remove_html_comments(text):
    return re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL) # Regex to match HTML comments

# Apply the function to the 'text' column
data['text'] = data['text'].apply(remove_inline_js_css)
# Apply the function to the 'text' column to remove HTML comments
data['text'] = data['text'].apply(remove_html_comments)
data.sample(10)

,text,label
598,OFFICE OF REVD=2EFR PETER CLARK DIRECTOR SPECI...,1
837,Ok,0
806,"ATTN:DIRECTOR/C E O, I know you will be surpr...",1
218,Jiloty Lauren C <JilotyLC@state.gov>Monday Sep...,0
501,Just rtnd your call she will be available on h...,0
59,Mr.Soo ChoCHO HUNG BANK.14 1-ka Namdaemun-ro C...,1
210,ok,0
783,"MR KENNY COLUBAN,NO,542 YOFF RUE,DAKAR-SENEGAL...",1
146,Will print for you in am too,0
422,FYI,0


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [8]:
# Your code
def remove_others(text):
    text = re.sub(r'[^A-Za-z\s]+', '', text, flags=re.DOTALL) # Regex to remove all the special characters and numbers
    text = re.sub(r'\b\w\b', '', text, flags=re.DOTALL) # Regex to remove all single characters
    text = re.sub(r' {2,}', ' ', text, flags=re.DOTALL) # Regex to substitute multiple spaces with single space
    text = re.sub(r'\bb', '', text, flags=re.DOTALL) # Regex to remove prefixed 'b'
    return text.lower()

data['text'] = data['text'].apply(remove_others)
data.sample(10)

,text,label
543,from lula davis regarding shannon and valenzue...,0
950,fyiwe have not announced the amount raised,0
913,hrodclintonemailcomsaturday january ammillscd...,0
262,plane crash web sitehttpewsbbccoukhiworldeurop...,1
497,mr david johnson liberte sicap cdakar senegale...,1
955,dear sirc got your contact in cause of serious...,1
274,greetings from mr abu msesaydear friendthrough...,1
90,there is very elegant picture of you on the fr...,0
782,great interview her on cnn or msnbc where she ...,0
117,book will e there in minutes,0


## Now let's work on removing stopwords
Remove the stopwords.

In [13]:
# Your code
from nltk.tokenize import word_tokenize

tokenized_data = [word_tokenize(doc) for doc in data["text"]]

no_stopwords_data = []
for email in tokenized_data:
    no_stopwords_email = []
    for word in email:
        if word not in stop_words:
            no_stopwords_email.append(word)
    no_stopwords_data.append(no_stopwords_email)

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [14]:
# Your code
from nltk.stem.wordnet import WordNetLemmatizer

lemmatized_data = [[WordNetLemmatizer().lemmatize(word) for word in sublist] for sublist in no_stopwords_data]


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [15]:
# Your code
from collections import Counter

flattened_data = [word for sublist in lemmatized_data for word in sublist]
word_counts = Counter(flattened_data)
top_10_words = word_counts.most_common(10)
print(top_10_words)

[('e', 1297), ('money', 924), ('account', 799), ('fund', 703), ('u', 666), ('country', 455), ('transaction', 412), ('million', 396), ('transfer', 393), ('ank', 392)]


### Let's divide the training and test set into two partitions

In [18]:
# Your code
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data["text"], data["label"], test_size=.2)

## Extra features

In [19]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

X_train['money_mark'] = X_train['preprocessed_text'].str.contains(money_simbol_list)*1
X_train['suspicious_words'] = X_train['preprocessed_text'].str.contains(suspicious_words)*1
X_train['text_len'] = X_train['preprocessed_text'].apply(lambda x: len(x)) 

X_test['money_mark'] = X_test['preprocessed_text'].str.contains(money_simbol_list)*1
X_test['suspicious_words'] = X_test['preprocessed_text'].str.contains(suspicious_words)*1
X_test['text_len'] = X_test['preprocessed_text'].apply(lambda x: len(x)) 

X_train.head()

KeyError: 'preprocessed_text'

## How would work the Bag of Words with Count Vectorizer concept?

In [20]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(lowercase=True, stop_words="english")
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [21]:
# Your code
vectorizer = TfidfVectorizer()

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)
display(X_train_vec.shape, X_test_vec.shape)

AttributeError: 'csr_matrix' object has no attribute 'lower'

## And the Train a Classifier?

In [ ]:
# Your code

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code